In [1]:
import numpy as np
import gurobi as gb
import matplotlib.pyplot as plt
import pandas as pd
# from scipy.optimize import minimize
from tqdm import tqdm
from numpy.random import normal, randint

GUROBI_TIMEOUT = 5 # in second

%matplotlib inline

In [5]:
travel_time = pd.read_csv('TimeOfTravel-1.csv')
n_weeks = travel_time.shape[0]
travel_time.head()

,week,travel_time
0,1,42
1,2,43
2,3,38
3,4,47
4,5,41


min |T-d|

min h where h = |T-d|

min h
s.t: 
- h+d <= T if d< T
- h-d <= -T if d>= T

In [19]:
# decision variables are (d, h1, h2, ..., h52)
obj = np.zeros(n_weeks+1)
obj[1:] = 1.0/n_weeks
lb = np.zeros(n_weeks+1)
lb[1:] = -np.inf # quantity printed needs to be non-negative, but profit on a given day could possibly be negative
rhs = np.zeros(2*n_weeks)
direction = np.array(['>']*(2*n_weeks))

In [22]:
A = np.zeros((2*n_weeks,n_weeks+1))
for r in range(n_weeks):
    A[2*r,[0,r+1]] = [1,1] # location 1 is q, location r+1 is this h
    rhs[2*r] = travel_time.travel_time[r]

    A[2*r+1,[0,r+1]] = [-1, 1]
    rhs[2*r+1] = -travel_time.travel_time[r]
A

array([[ 1.,  1.,  0., ...,  0.,  0.,  0.],
       [-1.,  1.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  1., ...,  0.,  0.,  0.],
       ...,
       [-1.,  0.,  0., ...,  0.,  1.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  1.],
       [-1.,  0.,  0., ...,  0.,  0.,  1.]])

In [23]:
spMod = gb.Model()
spMod_x = spMod.addMVar(len(obj),lb=lb) # tell the model how many variables there are
# spMod_x = spMod.addMVar(len(obj),lb=lb, vtype = ['I']+['C']*nd) # tell the model how many variables there are
# must define the variables before adding constraints because variables go into the constraints
spMod_con = spMod.addMConstrs(A, spMod_x, direction, rhs) # NAME THE CONSTRAINTS!!! so we can get information about them later!
spMod.setMObjective(None,obj,0,sense=gb.GRB.MINIMIZE) # add the objective to the model...we'll talk about the None and the 0

spMod.Params.OutputFlag = 0 # tell gurobi to shut up!!
spMod.optimize()


print(f'E(T):{spMod.objVal}')
print(f'd:{spMod_x.x[0]}')

E(T):5.078431372549019
d:44.0
